In [1]:
import pandas as pd
import numpy as np
import os 
import pandas as pds
from dask import dataframe
import re
import numpy as np
import seaborn as sbn
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

# 1. 불필요한 문자 제거하기

In [2]:
train = pd.read_csv("train.csv")
validation = pd.read_csv("validation_sample.csv")
test= pd.read_csv("test.csv")

In [3]:
lit = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
def mask(tt):
    tt=tt.apply(lambda x: re.sub(r'(\\n)',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[^a-zA-Zㄱ-ㅣ가-힣0-9:=\s\(\)./,\<\>]+',' ',x))
    #tt=tt.apply(lambda x: re.sub(r' ?(?P<note>[:=\(\)./,\<\>]) ?', ' \g<note> ', x))
    tt=tt.apply(lambda x: re.sub(r'[0-9]+',' ',x))
    tt=tt.apply(lambda x: re.sub(r"':/()",' ',x))
    tt=tt.apply(lambda x: re.sub(r':',' ',x))
    tt=tt.apply(lambda x: re.sub(r',',' ',x))
    # = tt.apply(lambda x: re.sub(r'(',' ',x))
    #t = tt.apply(lambda x: re.sub(r')',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ',x))
    for st in lit:
        st = " "+st + " "
        tt=tt.apply(lambda x: re.sub(st,' ',x))
    tt=tt.apply(lambda x: re.sub(r'\s+',' ',x))
    
    return tt

In [89]:
validation['pre_log'] = mask(validation.full_log)
train['pre_log'] = mask(train.full_log)
test['pre_log'] = mask(test.full_log)

KeyboardInterrupt: 

In [2]:
train = pd.read_csv("mask_train.csv")
valid = pd.read_csv("mask_vaild.csv")
test= pd.read_csv("mask_test.csv")

In [3]:
train

,id,level,full_log,pre_log,first
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err...",Sep localhost kibana type error timestamp Z ta...,...
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...,Feb localhost logstash INFO logstash outputs e...,...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err...",Jan localhost kibana type error timestamp Z ta...,...
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err...",Jan localhost kibana type error timestamp Z ta...,...
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...,type SYSCALL msg audit arch syscall success ye...,...
...,...,...,...,...,...
472967,472967,0,Feb 28 10:10:06 localhost logstash: 7738 error:,Feb localhost logstash error,
472968,472968,1,type=SYSCALL msg=audit(1611890993.458:321827):...,type SYSCALL msg audit arch syscall success ye...,...
472969,472969,0,"Oct 12 02:20:29 localhost kibana: {""type"":""log...",Oct localhost kibana type log timestamp Z tags...,
472970,472970,0,"Jan 15 09:02:43 localhost kibana: {""type"":""err...",Jan localhost kibana type error timestamp Z ta...,...


# 2. TRAIN에 있는 단어들 추출 


In [4]:
def count_word(data):
    tem = list(data['pre_log'].str.split(" "))
    all_word = []
    for word in tem:
        all_word.extend(word)
    words = pd.Series(all_word)
    return words.value_counts()

In [5]:
train_count = count_word(train)
test_count = count_word(test)

In [6]:
train_count[:50]

type             742245
audit            621461
error            547791
msg              502170
localhost        429812
kibana           423669
                 418171
connections      390510
No               377580
Living           362804
cap              361842
at               354980
js               354702
usr              323531
elasticsearch    319844
message          301162
pid              287580
system           271071
Error            265529
lib              252068
modules          243902
name             239434
node             236898
to               234380
share            233808
src              233423
process          228768
logstash         228362
next             226356
bin              224909
Jan              209292
unconfined       202264
fd               174349
connection       171539
timestamp        170854
Z                170564
tags             170189
ossec            154387
syscheckd        149400
object           134701
dev              134479
mode            

In [7]:
test_count

type         2227008
audit        1864487
error        1646310
msg          1506919
localhost    1290785
              ...   
caeafc             1
JAxx               1
HHk                1
febbaa             1
feebecc            1
Length: 18571, dtype: int64

In [12]:
total_word = set(list(set(test_count.index))+list(set(train_count.index)))

In [15]:
len(total_word)

21075

In [13]:
diff_word

{'fcbddf',
 'First',
 'fbbcf',
 'XB',
 'fbefecea',
 'dcfac',
 'OBp',
 'fffbcfc',
 'fbedefad',
 'bcfef',
 'Kme',
 'zJgHcBalm',
 'ebcc',
 'ecaed',
 'edfaf',
 'xwD',
 'beddd',
 'deeaea',
 'efcffb',
 'effeef',
 'WMI',
 'ginx',
 'fbfffd',
 'dUYifo',
 'dbgHcBYokOthzH',
 'aafee',
 'namereason',
 'eC',
 'ceabd',
 'gHcBYokOthzHMRwY',
 'Txk',
 'kCWf',
 'eafdd',
 'daedb',
 'bfccf',
 'dddda',
 'bcace',
 'fedeb',
 'ffcded',
 'Clock',
 'MEz',
 'ffceebffd',
 'fecddf',
 'RATEEST',
 'ddadff',
 'bfade',
 'fbdbce',
 'HUNTING',
 'why',
 'accca',
 'lzo',
 'mqdegHcBYokOthzHDQvw',
 'CHzm',
 'fcecbea',
 'fdcee',
 'fdbdbae',
 'accaae',
 'MgPTnU',
 'fadcbd',
 'fffecff',
 'mkzXgHcBalm',
 'arcfb',
 'DriverSearching',
 'fffdcee',
 'fdfcfa',
 'fabdcdb',
 'fffafd',
 'dkzjgHcBalm',
 'fdcfe',
 'held',
 'eedec',
 'ffeabd',
 'fbaccd',
 'mR',
 'dfabf',
 'ffffeaf',
 'hiYO',
 'ffefead',
 'efefa',
 'faeab',
 'ccec',
 'ccecbbb',
 'baffed',
 'ddceb',
 'cfbeabd',
 'SDKs',
 'dcffeac',
 'ecade',
 'fdbdbc',
 'abebe',
 'sBBs',
 'f

In [14]:
train_check = pd.DataFrame([train_count.index, train_count]).T
test_check= pd.DataFrame([test_count.index, test_count]).T

In [15]:
tt = test_check[test_check[0].isin(diff_word)]

In [28]:
def count(data):
    return len(data)
tt['len'] = tt[0].map(count)
tt

<ipython-input-28-104f998e706a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tt['len'] = tt[0].map(count)


,0,1,len
1386,Kill,197,4
1388,sacrifice,197,9
1599,ANOM,154,4
1818,sig,127,3
1820,ABEND,127,5
...,...,...,...
18565,gfNgincB,1,8
18566,fffdbeb,1,7
18567,ffabfd,1,6
18568,dfeacd,1,6


In [34]:
black_words = list(tt[(tt[1] >10) & (tt['len'] > 2)][0])
black_words

['Kill',
 'sacrifice',
 'ANOM',
 'sig',
 'ABEND',
 'gpgcheck',
 'prom',
 'hids',
 'intruder',
 'membership',
 'shipped',
 'irda',
 'PROMISCUOUS',
 'lex',
 'vstp',
 'tab',
 'runlevel',
 'capabilities',
 'become',
 'apparmor',
 'sophisticated',
 'binaries',
 'gains',
 'deviations',
 'including',
 'FILENAME',
 'locating',
 'Repositories',
 'Intrusion',
 'PACKAGENAME',
 'Certificates',
 'signed',
 'maintained',
 'defense',
 'approved',
 'includes',
 'Authority',
 'confining',
 'Verifying',
 'certificates',
 'indicates',
 'setperms',
 'Correct',
 'tampered',
 'gruop',
 'validates',
 'generous',
 'weakened',
 'owns',
 'Package',
 'Enabled',
 'Output',
 'Self',
 'authenticity',
 'prevention',
 'Remove',
 'Prevent',
 'instances',
 'intltoolize',
 'ntfs',
 'ntfscomp',
 'edff',
 'overall',
 'autodetect',
 'rmkeytab',
 'balance',
 'aedf',
 'idp',
 'regular',
 'recordmcount',
 'planes',
 'ebde',
 'livepatch',
 'opa',
 'ffcdca',
 'efeed',
 'xoffset',
 'dcee',
 'sec',
 'fbdef',
 'dlm',
 'vsock',
 'W

## black words  TEST 에 존재하는 열 확인해보기

In [58]:
def check7(data):
    tem = ""
    for i in black_words:
        i = " "+i+" "
        if i in data:
            tem = tem+i
        else:
            continue
            
    return tem

test['check'] = test['pre_log'].map(check7)

In [65]:
test['check'].value_counts()

                             1418017
 Kill  sacrifice                 181
 ANOM  sig  ABEND                127
 ANOM  prom  PROMISCUOUS          27
 vstp                             25
                              ...   
 become                            3
 shipped  lex                      3
 shipped                           2
 dcde  baac                        1
 capabilities                      1
Name: check, Length: 62, dtype: int64

In [60]:
del test['full_log']
del test['pre_log']

In [62]:
test.to_csv("test_for_auto.csv")

In [69]:
test['check'].value_counts()

                             1418017
 Kill  sacrifice                 181
 ANOM  sig  ABEND                127
 ANOM  prom  PROMISCUOUS          27
 vstp                             25
                              ...   
 become                            3
 shipped  lex                      3
 shipped                           2
 dcde  baac                        1
 capabilities                      1
Name: check, Length: 62, dtype: int64

In [71]:
test

,id,first,check
0,1000000,...,
1,1000001,...,
2,1000002,...,
3,1000003,...,
4,1000004,...,
...,...,...,...
1418911,2418911,...,
1418912,2418912,,
1418913,2418913,...,
1418914,2418914,...,


## 공통 단어 모아서 영단어 체크 dict
+ 1. 포함되어 있는 단어 모두 모아서 영단어 여부 체크
+ 2. map함수로 영단어 아니면 삭제

In [17]:
len(total_word)

21075

In [28]:
#영어 단어 + 2글자 이상인 경우만 True
from nltk.corpus import words
def check_words(word_list: list):
    re = [False] * len(word_list)
    for i,word in enumerate(word_list):
        if len(word) < 3:
            continue
        word = word.lower()
        if word in words.words():
            re[i] = True
           
        
        """
        else:
            word2 = " "+word+" "
            if word2 in train_bag_words:
                continue
                
            if word in words.words():
                re[i] = True
        """
    return re

word_list = ['promiscuous', 'ab', 'nihongo', 'abstract', 'pedo']

In [30]:
total_tf = check_words(total_word)

In [46]:
total_word = list(total_word)
for i in range(len(total_word)):
    total_word[i] = total_word[i].lower()
total_word

['',
 'methods',
 'jlxa',
 'fceda',
 'folu',
 'lzw',
 'fdaee',
 'fafcdde',
 'lockdev',
 'faffa',
 'qjts',
 'efcfacdf',
 'pn',
 'ffedf',
 'as',
 'gfp',
 'wcifs',
 'microcode',
 'yonb',
 'aebff',
 'fadfdaf',
 'isn',
 'repositories',
 'ntptime',
 'ffeddb',
 'bece',
 'cbb',
 'qhiv',
 'pv',
 'adwaita',
 'localtime',
 'xq',
 'adca',
 'iscsiadm',
 'exan',
 'bfd',
 'gconf',
 'confdata',
 'zl',
 'longer',
 'directories',
 'kannada',
 'colrm',
 'fbaacf',
 'faffef',
 'ffeddae',
 'fcddc',
 'bdp',
 'ccbce',
 'aaa',
 'xafdf',
 'cacad',
 'ffecf',
 'ffdbee',
 'dispose',
 'dbbdab',
 'ffebabde',
 'acbec',
 'vrkincb',
 'synchronous',
 'fddaed',
 'lycd',
 'bootchart',
 'dafc',
 'mangle',
 'ffcfc',
 'debacabb',
 'laptop',
 'macs',
 'isrunning',
 'boolinit',
 'supportassist',
 'ffdccc',
 'baeb',
 'partial',
 'ffefbe',
 'feebbc',
 'febaf',
 'rpclocator',
 'faefbda',
 'pps',
 'faebe',
 'fbcdfd',
 'fafbbcb',
 'pjnp',
 'fipvlan',
 'trespass',
 'ccdde',
 'saned',
 'sparc',
 'dvcont',
 'vinagre',
 'decdb',
 'too'

In [47]:
#단어 딕셔너리 만들어주기
isword = dict()
for i in range(len(total_word)):
    isword[total_word[i]] = total_tf[i]

In [48]:
isword

{'': False,
 'methods': False,
 'jlxa': False,
 'fceda': False,
 'folu': False,
 'lzw': False,
 'fdaee': False,
 'fafcdde': False,
 'lockdev': False,
 'faffa': False,
 'qjts': False,
 'efcfacdf': False,
 'pn': False,
 'ffedf': False,
 'as': False,
 'gfp': False,
 'wcifs': False,
 'microcode': False,
 'yonb': False,
 'aebff': False,
 'fadfdaf': False,
 'isn': False,
 'repositories': False,
 'ntptime': False,
 'ffeddb': False,
 'bece': False,
 'cbb': False,
 'qhiv': False,
 'pv': False,
 'adwaita': False,
 'localtime': False,
 'xq': False,
 'adca': False,
 'iscsiadm': False,
 'exan': False,
 'bfd': False,
 'gconf': False,
 'confdata': False,
 'zl': False,
 'longer': True,
 'directories': False,
 'kannada': False,
 'colrm': False,
 'fbaacf': False,
 'faffef': False,
 'ffeddae': False,
 'fcddc': False,
 'bdp': False,
 'ccbce': False,
 'aaa': False,
 'xafdf': False,
 'cacad': False,
 'ffecf': False,
 'ffdbee': False,
 'dispose': True,
 'dbbdab': False,
 'ffebabde': False,
 'acbec': False,
 

In [52]:
# 영어 단어인지 확인해주는 dic

def checking(data):
    re = [False] * len(data)
    for i,word in enumerate(data):
        if isword[word]:
            re[i] = True
    
    return re


def cutt(data):
    data = data.lower()
    splited = data.split(" ")
    check = checking(splited)
    c = np.array(splited)
    a = c[check]
    real_words = list(c[check])
    tem = " ".join(real_words)
    return tem


In [55]:
train['cut'] = train['pre_log'].map(cutt)

In [57]:
test['cut'] = test['pre_log'].map(cutt)

In [56]:
valid['cut']

0    type promiscuous audit dev old gid type audit ...
1    result scan content title prevent log with emp...
2    kernel out memory kill process probe score sac...
Name: cut, dtype: object

In [44]:
vali['ANOM']

NameError: name 'vali' is not defined

In [59]:
del train['full_log']
del test['full_log']

del train['pre_log']
del test['pre_log']

del train['first']
del test['first']

In [61]:
train.to_csv("onlyword_train.csv",index =False)
test.to_csv("onlyword_test.csv",index =False)